In [1]:
import jsonlines
import pandas as pd

import requests
import json
import time
import pandas as pd
from tqdm import tqdm
import os  # os 모듈 추가|


In [2]:
input_folder = '../국방부훈령QA생성_llm_train'
input_files = [os.path.join(input_folder, file) for file in os.listdir(input_folder) if file.endswith('.jsonl')]

In [3]:
input_files[1]

'../국방부훈령QA생성_llm_train/5ㆍ18 민주화운동 진상규명조사위원회 설립준비단 설치 및 운영에 관한 규정.jsonl'

In [4]:

## system prompt와 model 정의
model = 'gpt-4o'

system_prompt = '''
## 역할
너는 규정에 대한 질의를 할 수 있는 질문생성기야. 대한민국 군인이라 생각하고 제공하는 조항에 대해 질문을 할 수 있어야 해. 그리고 그 질문에 대해 명확한 답변을 제공할 수 있어야 한다.

## 지시
1.  내가 규정의 한 조항을 알려줄테니, 그 조항을 바탕으로 대한민국 군인이 궁금해 할 법한 질문을 생성해줘. 

## 조건
- 질문의 개수는 최소 1개, 최대 5개이며, 질문의 내용이 중복되지 않아야 한다.
- 질문하는 말투는 "~~이 뭐야?" "~~이 뭡니까?" "~~이 궁금합니다" 등 다양해야한다.
- 반드시 규정 안에서만 답변할 수 있을만한 질문이어야 한다. 
- 답변은 친절하게 말해야하며, 반드시 조항 안에 있는 내용만을 바탕으로 답변해야 한다.
- 답변은 "~~에 대한 질문을 주셨군요". "~~ 에 대해 답변드리겠습니다"와 답변 전 공감이 들어가야 한다.
- 삭제 등 질문을 추출할 수 없는 정보는 "질문 없음"이라고 표기한다.

## 포맷
- {질문1} | {답변1}
- {질문2} | {답변2}
...

## 예시
부대에서 술을 마셔도 되나요? | 부대에서 음주가 가능한지 질문하셨군요. \n 술과 담배는 허가된 시간과 장소에서 마시거나 피워야 합니다. 따라서 부대에서 허가된 장소에서 음주하세요. 
부대에서 담배 펴도 됨? | 부대 내 흡연 가능 여부에 대해 답변드리겠습니다. \n 술과 담배는 허가된 시간과 장소에서만 마시거나 피울 수 있습니다. 부대의 허가된 장소에서만 흡연이 가능합니다. 
부대에서 술을 마셔도 됩니까? | 부대 내 음주 가능 여부에 대해 말씀드리겠습니다. \n 술과 담배는 허가된 시간과 장소에서 마시거나 피워야 합니다. 따라서 부대에서 허가된 장소에서 음주하세요.
'''

## 모델 parameter 지정
temperature = 0
top_p = 0.1
presence_penalty = 0
frequency_penalty = 0
max_tokens = 4096

In [5]:
input_file = input_files[40]
data = []
# JSONL 파일에서 데이터를 읽어 리스트에 저장합니다.
with jsonlines.open(input_file) as reader:
    for obj in reader:
        print(obj)
        data.append({"user": obj["text"]})

data

[]

In [6]:
army_df = pd.DataFrame(columns = ['model','system','document','user'])

## 배치 전용 파일 만들기
for input_file in tqdm(input_files):
    ## 파일 이름 추출
    file_name = os.path.splitext(os.path.basename(input_file))[0]
    
    data = []
    # JSONL 파일에서 데이터를 읽어 리스트에 저장합니다.
    with jsonlines.open(input_file) as reader:
        for obj in reader:
            data.append({"user": obj["text"]})
    
    if len(data) == 0:
        print(f"{input_file} 파일이 비었습니다.")
        continue    
     
    # pandas DataFrame으로 변환
    df = pd.DataFrame(data)
    df = df[~df['user'].str.contains("부칙 ")]
    
    ## 파일 이름과 모델 이름, system prompt 정의
    df['document'] = file_name
    df['system'] = system_prompt
    df['model'] = model
    
    army_df = pd.concat([army_df, df], axis = 0).reset_index(drop=True)

 37%|███▋      | 86/232 [00:00<00:00, 280.28it/s]

../국방부훈령QA생성_llm_train/국방부 공무원의 부동산 신규취득 제한에 관한 지침.jsonl 파일이 비었습니다.
../국방부훈령QA생성_llm_train/국방전력발전업무훈령.jsonl 파일이 비었습니다.
../국방부훈령QA생성_llm_train/국방조직 및 정원 관리 훈령.jsonl 파일이 비었습니다.


100%|██████████| 232/232 [00:00<00:00, 257.81it/s]


In [7]:
## model, system prompt, user prompt 지정
model = army_df['model']
system = army_df['system']
user = army_df['user']

## 배치파일 만들기
n = len(user)
result = []

for i in tqdm(range(n)):
    task = {
            "custom_id": f"task{i}",
            "method": "POST",
            "url": "/v1/chat/completions",
            "body": {
                # This is what you would have in your Chat Completions API call
                "model": model[i],
                "messages": [
                    {
                        "role": "system",
                        "content": system[i]
                    },
                    {
                        "role": "user",
                        "content": user[i]
                    }
                ],
                "temperature": temperature,
                "top_p": top_p,
                "presence_penalty": presence_penalty,
                "frequency_penalty": frequency_penalty,
                "max_tokens": max_tokens,
            }
        }
    # print(task)
    result.append(task)

  0%|          | 0/8508 [00:00<?, ?it/s]

100%|██████████| 8508/8508 [00:00<00:00, 56356.73it/s]


In [9]:
jsonl_name = f"국방부훈령_JSONL_RESULT.jsonl"

with open(jsonl_name, 'w', encoding='utf-8') as file:
    for obj in result:
        file.write(json.dumps(obj, ensure_ascii=False) + '\n')

In [24]:
input_file = './부대관리 훈령.jsonl'   # JSONL 파일 경로
# 파일명 추출 (확장자 제외)
file_name = os.path.splitext(os.path.basename(input_file))[0]
# output_file = './부대관리 훈령.csv'

In [27]:
army_df.to_excel('./국방부훈령_batch_input.xlsx', index = False)

In [31]:
# data = []

# # JSONL 파일에서 데이터를 읽어 리스트에 저장합니다.
# with jsonlines.open(input_file) as reader:
#     for obj in reader:
#         data.append({"user": obj["text"]})

# # pandas DataFrame으로 변환
# df = pd.DataFrame(data)

# # DataFrame을 CSV 파일로 저장
# # df.to_csv(output_file, index=False, encoding = "euc-kr")

# ## system prompt와 model 정의
# model = 'gpt-4o'

# system_prompt = '''
# ## 역할
# 너는 규정에 대한 질의를 할 수 있는 질문생성기야. 대한민국 군인이라 생각하고 제공하는 조항에 대해 질문을 할 수 있어야 해. 그리고 그 질문에 대해 명확한 답변을 제공할 수 있어야 한다.

# ## 지시
# 1.  내가 규정의 한 조항을 알려줄테니, 그 조항을 바탕으로 대한민국 군인이 궁금해 할 법한 질문을 생성해줘. 

# ## 조건
# - 질문의 개수는 최소 1개, 최대 5개이며, 질문의 내용이 중복되지 않아야 한다.
# - 질문하는 말투는 "~~이 뭐야?" "~~이 뭡니까?" "~~이 궁금합니다" 등 다양해야한다.
# - 반드시 규정 안에서만 답변할 수 있을만한 질문이어야 한다. 
# - 답변은 친절하게 말해야하며, 반드시 조항 안에 있는 내용만을 바탕으로 답변해야 한다.
# - 답변은 "~~에 대한 질문을 주셨군요". "~~ 에 대해 답변드리겠습니다"와 답변 전 공감이 들어가야 한다.
# - 삭제 등 질문을 추출할 수 없는 정보는 "질문 없음"이라고 표기한다.

# ## 포맷
# - {질문1} | {답변1}
# - {질문2} | {답변2}
# ...

# ## 예시
# 부대에서 술을 마셔도 되나요? | 부대에서 음주가 가능한지 질문하셨군요. \n 술과 담배는 허가된 시간과 장소에서 마시거나 피워야 합니다. 따라서 부대에서 허가된 장소에서 음주하세요. 
# 부대에서 담배 펴도 됨? | 부대 내 흡연 가능 여부에 대해 답변드리겠습니다. \n 술과 담배는 허가된 시간과 장소에서만 마시거나 피울 수 있습니다. 부대의 허가된 장소에서만 흡연이 가능합니다. 
# 부대에서 술을 마셔도 됩니까? | 부대 내 음주 가능 여부에 대해 말씀드리겠습니다. \n 술과 담배는 허가된 시간과 장소에서 마시거나 피워야 합니다. 따라서 부대에서 허가된 장소에서 음주하세요.
# '''

# df['system'] = system_prompt
# df['model'] = model

# ## model, system prompt, user prompt 지정
# model = df['model']
# system = df['system']
# user = df['user']

# ## 모델 parameter 지정
# temperature = 0
# top_p = 0.1
# presence_penalty = 0
# frequency_penalty = 0
# max_tokens = 4096

# ## 배치파일 만들기
# n = len(user)
# result = []

# for i in tqdm(range(n)):
#     task = {
#             "custom_id": f"task{i}",
#             "method": "POST",
#             "url": "/v1/chat/completions",
#             "body": {
#                 # This is what you would have in your Chat Completions API call
#                 "model": model[i],
#                 "messages": [
#                     {
#                         "role": "system",
#                         "content": system[i]
#                     },
#                     {
#                         "role": "user",
#                         "content": user[i]
#                     }
#                 ],
#                 "temperature": temperature,
#                 "top_p": top_p,
#                 "presence_penalty": presence_penalty,
#                 "frequency_penalty": frequency_penalty,
#                 "max_tokens": max_tokens,
#             }
#         }
#     # print(task)
#     result.append(task)
    
# jsonl_name = f"{file_name}_JSONL_RESULT.jsonl"
# with open(jsonl_name, 'w', encoding='utf-8') as file:
#     for obj in result:
#         file.write(json.dumps(obj, ensure_ascii=False) + '\n')
